## C-Trap Unzippong Pre-Processing

In [ ]:
from scipy.optimize import curve_fit
from scipy import signal
from scipy import stats
import lumicks.pylake as lk
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import csv
import IPython.display as ipy
import warnings
import math
import seaborn
import statistics
import re
import psutil
import gc

# Functions

In [ ]:
#checks for a baseline mapping file and baseline file corresponding to the specified curve
def has_baseline(folder, curve):
    folder += '/baseline/'
    base = 'baselines.csv'
    
    #check for baseline folder
    if not os.path.exists(folder):
        return -1
    
    #find baseline mapping file
    for filename in os.listdir(folder):
        if os.path.isfile(folder+base) and base in filename:
            baselines = pd.read_csv(folder+filename)
            break
    
    #find corresponding baseline for the curve
    for col in baselines.columns:
        if int(curve) in baselines[col].values or float(curve) in baselines[col].values:
            return int(col)
    return -1

In [ ]:
#From a filename, data (force, trap positions, and time) are extracted from Lumicks H5 files and downsampled to
#a desired frequncy. Addresses some bugs in data export that arose in different versions of Lumicks BlueLake software
def h5_extract(f, compress = 78125, shift = 0):
    
    #import file
    file = lk.File(f)
    
    try:
    #extract high frequency force and time (F, t)
        F2x = file["Force HF"]["Force 2x"]
        F2y = file["Force HF"]["Force 2y"]
        F2 = F2x.data
        F1x = file["Force HF"]["Force 1x"]
        F1y = file["Force HF"]["Force 1y"]
        F1 = F1x.data
        t = F2x.seconds
    except:
        print('Issue with forces, not downsampled')
        return np.empty(0), np.empty(0), np.empty(0), np.empty(0), np.empty(0), np.empty(0), np.empty(0), np.empty(0), np.empty(0)
    
    #LUMICKS BUG: STARTING AUGUST 2023 position1x data was stored in position1Y and vice versa 
    try:
        #extract high frequency trap position (p)
        #position1x = file["Trap position"]['1X']
        position1x = file["Trap position"]['1Y']
        p1x = position1x.data
        position2x = file["Trap position"]['2Y']
        p2x = position2x.data
        position1y = file["Trap position"]['1X']
        p1y = position1y.data
        position2y = file["Trap position"]['2X']
        p2y = position2y.data
        #position2x = file["Trap position"]['2X']
    except:
        print('Issue with trap positions, not downsampled')
        return np.empty(0), np.empty(0), np.empty(0), np.empty(0), np.empty(0), np.empty(0), np.empty(0), np.empty(0), np.empty(0)
    
    #down sample high frequency data to usable frequency
    #at some point Lumicks began exporting position2x and position2y as a single 0, not an array
    #0 array created in a later function
    compressedF2y = F2y.downsampled_to(compress, method="force")
    compressedF1y = F1y.downsampled_to(compress, method="force")
    compressedF1x = F1x.downsampled_to(compress, method="force")
    compressedF2x = F2x.downsampled_to(compress, method="force")
    compressedP1x = position1x.downsampled_to(compress, method="force")
    compressedP1y = position1y.downsampled_to(compress, method="force")
    #compressedP2x = position2x.downsampled_to(compress, method="force")
    #compressedP2y = position2y.downsampled_to(compress, method="force")

    cP1x = compressedP1x.data
    cP1y = compressedP1y.data
    mF1x = compressedF1x.data*-1
    mF2x = compressedF2x.data
    mt = compressedF1x.seconds

    mF2y = compressedF2y.data
    mF1y = compressedF1y.data*-1
    
    print('compressed from ' + str(t.size) + ' to ' + str(mt.size))
    
    return mt, mF1x, mF1y, cP1x, mF2x, mF2y, cP1y, p2x, p2y

In [ ]:
#return array of indeces for baseline subtraction. Finds index of closest baseline trap position to data trap position
#adapted from: https://stackoverflow.com/questions/2566412/find-nearest-value-in-numpy-array
def find_nearest(array1, array2):
    array1 = np.asarray(array1)
    array2 = np.asarray(array2)
    idx = []
    for v in array1:
        idx.append(np.abs(array2 - v).argmin())
    return idx

In [ ]:
#given which variable to search, uses sliding window to find a specified number of peaks/valleys or
#prompt the user (if peaks == -1) for a number of peaks/valleys. Can be used to find cycle starts and stops and/or force peaks
#'trap' is default and finds cycles, 'f' finds force peaks
def find_peaks(df, date, curve, cycle, grna = -1, var = 'trap', sign = '+', num_peaks = -1, interval = 1000, peak_dict=None):#, peak_types=[]):
    
    #initialize variables
    possiblex = []
    possibley = []
    num_in = -1
    start = 0
    label = 'peak'
    if sign == '-':
        label = 'valley'

    #confirm variable types are valid
    while var != 'trap' and var != 'f':
        print('Bad entry for "var", enter "trap" or "f"')
        var = input()
        
    while sign != '+' and sign != '-':
        print('Bad entry for "sign", enter "+" or "-" for peaks or valleys, respectively')
        var = input()

    #if number of peaks not specified, plot and ask. If called or answered with 0, exit
    if num_peaks == -1:
        fig, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize = (15, 5))
        ax1.plot(df['trap_position 1x'], df['force baseline 2x'])
        ax2.plot(df['time (seconds)'], df['force baseline 2x'])
        ax3.plot(df['time (seconds)'], df['trap_position 1x'])
        ax1.set(xlabel = 'Extension (um)', ylabel = 'Force (pN)', title = 'Curve ' + str(curve))
        ax2.set(xlabel = 'Time (seconds)', ylabel = 'Force (pN)', title = 'Curve ' + str(curve))
        ax3.set(xlabel = 'Time (seconds)', ylabel = 'Trap position (um)', title = 'Curve ' + str(curve))
        plt.show()
        if var == 'trap':
            print('How many cycles? (out and back is 1, just out is 0.5)')
        elif var == 'f':
            print('How many force rips?')
        num_in = input()
        num_peaks = num_in
        ipy.clear_output()
    else:
        num_in = num_peaks
    
    #exit function if no peaks to be found
    if num_peaks == '0' or num_peaks == 0:
        return [], [], 0, peak_dict
    
    #adjust number of peaks if non-integer value (for example,0.5 means only forward, no reverse)
    if sign == '-' and var == 'trap':
        num_peaks = math.floor(float(num_peaks)) + 1
        search_max = 2
    elif sign == '+' and var == 'trap':
        num_peaks = math.ceil(float(num_peaks))
        search_max = 2
    elif var == 'f':
        num_peaks = int(num_peaks)
        search_max = 1.5
    
    #loop through until user satisfied
    answer = 0
    while answer == 0:
        
        #initialize varibales
        possiblex = []
        possibley = []
        start = 0
        
        #search along the curve in every window and find the max or min in the window (trap position or force)
        while start < len(df.index)-1:
            stop = start + interval
            stop = min(stop, len(df.index)-1)
            df2 = df.iloc[start:stop, :]

            #to split into cycles, find local maxima and minima for trap position
            if var == 'trap':
                if sign == '+':
                    possibley.append(max(df2['trap_position 1x']))
                    possiblex.append(df2['trap_position 1x'].idxmax())
                elif sign == '-':
                    possibley.append(min(df2['trap_position 1x']))
                    possiblex.append(df2['trap_position 1x'].idxmin())
            
            #to find force peaks, find local maxima
            elif var == 'f':
                if sign == '+':
                    if df['trap_position 1x'].iloc[df2['force baseline 2x'].idxmax()] < search_max:
                        possibley.append(max(df2['force baseline 2x']))
                        possiblex.append(df2['force baseline 2x'].idxmax())
                elif sign == '-':
                    possibley.append(min(df2['force baseline 2x']))
                    possiblex.append(df2['force baseline 2x'].idxmin())
            start += interval

        #order the list of possible peaks, remove duplicates, and select highest
        rev = True
        if sign == '-':
            rev = False
        
        #makes sure peaks are at least 5 seconds appart
        if var == 'trap':
            l = len(possiblex)
            for i in range(0, l-1):
                check = l-i-1
                #print(check)
                if abs(df['time (seconds)'].iloc[possiblex[check]] - df['time (seconds)'].iloc[possiblex[check-1]]) < 2:
                    if (sign == '-' and possibley[check] > possibley[check-1]) or (sign == '+' and possibley[check] < possibley[check-1]):
                        del possiblex[check]
                        del possibley[check]
                    else:
                        del possiblex[check-1]
                        del possibley[check-1]
        
        #Too debug issue where first cycle minima missed
        if sign == '-' and len(possiblex) < num_peaks:
            possibley.append(df2.iloc[0]['force baseline 2x'])
            possiblex.append(0)
        
        possibley, possiblex = zip(*sorted(zip(possibley, possiblex), reverse=rev))
        
        #pick top (or bottom) peaks
        peaksx = possiblex[0:num_peaks]
        peaksy = possibley[0:num_peaks]
        
        peaksx, peaksy = zip(*sorted(zip(peaksx, peaksy)))

        #have user validate the peaks and save if correct
        #ipy.clear_output()
        fig, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize = (15, 5))
        ax1.plot(df['trap_position 1x'], df['force baseline 2x'])
        ax2.plot(df['time (seconds)'], df['force baseline 2x'])
        ax3.plot(df['time (seconds)'], df['trap_position 1x'])
        ax1.set(xlabel = 'Extension (um)', ylabel = 'Force (pN)', title = date + ' Curve ' + str(curve) + ' Cycle ' + str(cycle))
        ax2.set(xlabel = 'Time (seconds)', ylabel = 'Force (pN)', title = date + ' Curve ' + str(curve) + ' Cycle ' + str(cycle))
        ax3.set(xlabel = 'Time (seconds)', ylabel = 'Trap position (um)', title = 'Curve ' + str(curve))
        for i in range(0, len(peaksy)):
            if var == 'f':
                f = df['force baseline 2x'].iloc[peaksx[i]]
                p = df['trap_position 1x'].iloc[peaksx[i]]
                d = df['trap_position 1x'].iloc[peaksx[i]]
                ax1.axhline(y=f, color='r')
                ax1.axvline(x=d, color='r')
                ax2.axhline(y=f, color='r')
                ax3.axhline(y=p, color='r')
                print(label + ' ' + str(i+1) + ' is at ' + str(f) + ' pN, ' + str(d) + ' um extension, ' + str(p) + ' trap position')
            if var == 'trap':
                d = df['trap_position 1x'].iloc[peaksx[i]]
                t = df['time (seconds)'].iloc[peaksx[i]]
                p = df['trap_position 1x'].iloc[peaksx[i]]
                ax1.axvline(x=d, color='r')
                ax2.axvline(x=t, color='r')
                ax3.axvline(x=t, color='r')
                print(label + ' ' + str(i+1) + ' is at ' + str(d) + ' um extension, ' + str(t) + " seconds, and " + str(p) + ' trap position')
        plt.show()
        print('Do ' + label + 's look correct? yes = 1, no = 0')
        answer = int(input())
        if answer == 0 and var == 'trap':
            print('new window size? was ' + str(interval))
            interval = int(input())
            ipy.clear_output()
        elif answer == 0 and var == 'f':
            print('New maximum distance to search? Was ' + str(search_max) + ' um')
            search_max = float(input())
            print('new window size? was ' + str(interval))
            interval = int(input())
            ipy.clear_output()
        elif answer ==1 and var == 'f' and peak_dict != None:
            for i in range(0, len(peaksy)):
                
                peak_dict['Date'].append(date)
                peak_dict['Curve'].append(curve)
                peak_dict['Cycle'].append(cycle)
                peak_dict['Force'].append(peaksy[i])
                peak_dict['Index'].append(peaksx[i])
                peak_dict['Trap Position'].append(df['trap_position 1x'].iloc[peaksx[i]])
                peak_dict['Distance'].append(df['trap_position 1x'].iloc[peaksx[i]])
                peak_dict['Time'].append(df['time (seconds)'].iloc[peaksx[i]])
                peak_dict['gRNA'].append(grna)
                
                print('What Cas9 site is peak ' + str(i+1) + ' at? (0 = WT, 1 = MM, 2 = Off)')
                peakt = int(input())
                if peakt == 1:
                    print('How many mismatches?')
                    peak_dict['Mismatches'].append(int(input()))
                    peak_dict['Mismatch Site'].append(1)
                    peak_dict['WT Site'].append(0)
                    peak_dict['Off Site'].append(0)
                    print('Did peak slide or is sliding? (0 = no, 1 = yes)')
                    peak_dict['Slid'].append(int(input()))
                else:
                    peak_dict['Mismatch Site'].append(0)
                    peak_dict['Mismatches'].append(-1)
                    peak_dict['Slid'].append(0)
                    
                if peakt == 0:
                    peak_dict['WT Site'].append(1)
                    peak_dict['Off Site'].append(0)
                elif peakt == 2:
                    peak_dict['WT Site'].append(0)
                    peak_dict['Off Site'].append(1)
                    

    ipy.clear_output()
    return peaksx, peaksy, num_in, peak_dict

In [ ]:
#functions that imports all csv files in a specified directory and returns a list of filenames 

def select_files(date, file_type, directory, selections = [], cycles = False):
    
    #initialize variables needed for selection
    folder = directory + date
    types = ['h5', 'down', 'calibration']
    if file_type in types:
        types.remove(file_type)
    out_parts = types
    file_list=[]
    if cycles:
        folder += '/cycles'

    #loop through specified folder and append all desired files to list
    for filename in os.listdir(folder):
        fi = os.path.join(folder, filename)
        
        #by default include; if no type provided, include all
        check = True
        if file_type == '':
            if os.path.isfile(fi) and check:
                file_list = np.append(file_list, fi)
                continue

        #make sure the correct file type
        if file_type not in fi:
            continue
        for part in out_parts:
            if part in fi:
                check = False
        if check == False:
            continue
                
        #if list of curves specified, make sure it's a desired one
        #if there's a list, make sure it's in it
        if selections:
            curve = int(extract_curve_number(fi))
            if cycles:
                cycle = int(extract_cycle_number(fi))
            check = False
            for s in selections:
                if type(s) == tuple and s[0] == curve and s[1] == cycle:
                    check = True
                elif s == curve:
                    check = True
                
        if os.path.isfile(fi) and check:
            file_list = np.append(file_list, fi)

    file_list = np.sort(file_list)
    
    return file_list

In [ ]:
# function to plot a list of input files on top of eachother and return the resulting figure
#accepts arguments for type of plots (d = force v extension, t = force v time, d/t = both side by side,
#d/t/p = both side by side + position vs time)
#and a title and legend
 
def plot_curves(file_list, plot='d/t', ftitle = '', legend=True, font=16, color='', smooth=False, xlim=False):
    
    #initialize list of curves for legend
    curves = []        
    
    #create specified figure type with labels
    if plot == 'd/t':
        fig, [ax1, ax2] = plt.subplots(1, 2, figsize = (15, 5))
        ax1.set(xlabel = 'Extension (um)', ylabel = 'Force (pN)', title = ftitle)
        ax2.set(xlabel = 'Time (seconds)', ylabel = 'Force (pN)', title = ftitle)
    elif plot == 'd/t/p':
        fig, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize = (15, 5))
        ax1.set(xlabel = 'Extension (um)', ylabel = 'Force (pN)', title = ftitle)
        ax2.set(xlabel = 'Time (seconds)', ylabel = 'Force (pN)', title = ftitle)
        ax3.set(xlabel = 'Time (seconds)', ylabel = 'Trap Position (um)', title = ftitle)
    elif plot == 'd':
        fig = plt.figure()
        plt.xlabel('Extension (um)', fontsize=font)
        plt.ylabel('Force (pN)', fontsize=font)
        plt.title(ftitle, fontsize=font)
    elif plot == 't':
        fig = plt.figure(figsize=(12, 4))
        plt.xlabel('Time (seconds)', fontsize=font)
        plt.ylabel('Force (pN)', fontsize=font)
        plt.title(ftitle, fontsize=font)
    
    #loop through files
    for f in file_list:
        #check if file and import if so
        if os.path.isfile(f):
            curve = extract_curve_number(f)
            curves = np.append(curves, curve)
            df = pd.read_csv(f)
        
            #plot desired plot type
            if color=='':

                if plot == 'd/t':
                    ax1.plot(df['extension'], df['force average'], alpha=0.5)
                    ax2.plot(df['time (seconds)'], df['force average'], alpha =0.5)
                if plot == 'd/t/p':
                    ax1.plot(df['extension'], df['force average'], alpha=0.5)
                    ax2.plot(df['time (seconds)'], df['force average'], alpha =0.5)
                    ax3.plot(df['time (seconds)'], df['trap_position 1x'], alpha=0.5)
                elif plot == 'd':
                    if smooth:
                        plt.plot(df['extension'], df['force average'], alpha=0.5, color='gray')
                        df['y_smoothed'] = df['force average'].rolling(window=smooth).mean()
                        plt.plot(df['extension'], df['y_smoothed'], alpha=1, color='red')
                    else:
                        plt.plot(df['extension'], df['force average'], alpha=0.5)
                elif plot == 't':
                    if smooth:
                        plt.plot(df['time (seconds)'], df['force average'], alpha=0.5, color='gray')
                        df['y_smoothed'] = df['force average'].rolling(window=smooth).mean()
                        plt.plot(df['time (seconds)'], df['y_smoothed'], alpha=1, color='red')
                    else:
                        plt.plot(df['time (seconds)'], df['force average'], alpha=0.5)
            else:
                if plot == 'd/t':
                    ax1.plot(df['distance (um)'], df['force 2x (pN)'], alpha=0.5, color=color)
                    ax2.plot(df['time (seconds)'], df['force 2x (pN)'], alpha =0.5, color=color)
                elif plot == 'd':
                    plt.plot(df['distance (um)'], df['force 2x (pN)'], alpha=0.5, color=color)
                elif plot == 't':
                    plt.plot(df['time (seconds)'], df['force 2x (pN)'], alpha=0.5, color=color)
                
    #add the legend
    if legend == True:
        plt.legend(curves)
    elif legend != False:
        plt.legend(legend)
        
    return fig

In [ ]:
def extract_curve_number(filename):
    # Regular expression pattern to match "Curve" followed by 1 to 3 digits
    pattern = r"Curve \s*(\d{1,3})"
    
    # Search for the pattern in the filename
    match = re.search(pattern, filename)
    
    if match:
        # Extract the matched number
        curve_number = match.group(1)
        # Add preceding zero if it's a single digit number
        if len(curve_number) == 1:
            curve_number = '0' + curve_number
        return curve_number
    else:
        return None  # Return None if no match is found

In [ ]:
def extract_cycle_number(filename):
    # Regular expression pattern to match "Curve" followed by 1 to 3 digits
    pattern = r"Cycle \s*(\d{1,3})"
    
    # Search for the pattern in the filename
    match = re.search(pattern, filename)
    
    if match:
        # Extract the matched number
        curve_number = match.group(1)
        # Add preceding zero if it's a single digit number
        if len(curve_number) == 1:
            curve_number = '0' + curve_number
        return curve_number
    else:
        return None  # Return None if no match is found

In [ ]:
def extract_stiffness(date, curve, trap, directory, var = 'kappa', suffix = ''):
    #find calibration files for particular date that are for the correct trap
    f = []
    cals = select_files(date, 'calibration', directory=directory)
    for cal in cals:
        if trap in cal:
            f.append(cal)
    
    #if there are more than 1 (two condition day) require it have the correct "suffix"
    if len(f) > 1:
        f2 = []
        for file in f:
            if suffix in file:
                f2.append(file)
        f = f2

    #open the calibration file
    if os.path.isfile(f[0]) and len(f) == 1:
        df = pd.read_csv(f[0])
        if var == 'kappa':
            return df.loc[df['curve'] == curve, 'kappa (pN/nm)'].values[0]
        elif var == 'r':
            return (df.loc[df['curve'] == curve, 'Bead diameter (um)'].values[0])/2
        else:
            return -1

# 1) PC VERSION: Extract fitting parameters

In [ ]:
#PyLake versions and Mac vs PC seem to return different calibration objects

#Fill in filepath, date, file selections and suffix (if desired to add to distinguish conditions from same date)

#File path is assumed to lead to subfolders of dates in the format YY_MM_DD and end in /
#Only selected files will be processed; an empty list ([]) will be treated as files being selected
directory = ""
date = ''
selections = []
suffix = ''

folder = directory+date

dates = date.split('_')
YY = dates[0]
MM = dates[1]
DD = dates[2]

#gather files
file_list = select_files(date, 'h5', selections = selections, directory=directory)
export = ["1x", '1y', '2x', '2y']

i = 0
for x in export:
    df = pd.DataFrame()
    name = date + suffix + '_' + x + '_calibration_stats.csv'
    fname = os.path.join(folder, name)
    for f in file_list:
        file = lk.File(f)
        if x == '1x':
            data = file.force1x.calibration[0]
        elif x == '2x':
            data = file.force2x.calibration[0]
        elif x == '1y':
            data = file.force1y.calibration[0]
        elif x == '2y':
            data = file.force2y.calibration[0]
        curve = extract_curve_number(f)
        data['date'] = date
        data['curve'] = int(curve)
        data['fname'] = DD + MM + YY + 'N' + curve + ".csv"
        row_df = pd.DataFrame(data, index=[i])
        df = pd.concat([df, row_df], ignore_index=True)
        i += 1
        del file
        gc.collect()
    
    curve_col = df.pop("curve")
    df.insert(0, "curve", curve_col)
    fname_col = df.pop("fname")
    df.insert(0, "fname", fname_col)
    date_col = df.pop("date")
    df.insert(0, "date", date_col)
    df.to_csv(fname)
    print(x + ' done')

# 1) MAC VERSION: Extract fitting parameters

In [ ]:
#PyLake versions and Mac vs PC seem to return different calibration objects

#Fill in filepath, date, file selections and suffix (if desired to add to distinguish conditions from same date)

#File path is assumed to lead to subfolders of dates in the format YY_MM_DD and end in /
#Only selected files will be processed; an empty list ([]) will be treated as files being selected
directory = ""
date = ''
selections = []
suffix = ''

folder = os.path.join(directory, date)

YY, MM, DD = date.split('_')

# === Gather files ===
file_list = select_files(date, 'h5', selections=selections, directory=directory)
export = ["1x", "1y", "2x", "2y"]

# === Process each force channel ===
for x in export:
    all_rows = []

    for f in file_list:
        file = lk.File(f)

        # Select calibration object
        if x == '1x':
            data = file.force1x.calibration[0]
        elif x == '1y':
            data = file.force1y.calibration[0]
        elif x == '2x':
            data = file.force2x.calibration[0]
        elif x == '2y':
            data = file.force2y.calibration[0]

        # Extract all public attributes (avoid methods and private)
        data_dict = {
            k: getattr(data, k)
            for k in dir(data)
            if not k.startswith("_") and not callable(getattr(data, k))
        }

        # Convert to wide DataFrame (one row)
        df_wide = pd.DataFrame([data_dict])
        # Each row's dict becomes columns; keeps any other columns you already had
        df_wide = pd.json_normalize(df_wide.pop("data"))


        # Add metadata columns
        curve = extract_curve_number(f)
        df_wide["curve"] = int(curve)
        df_wide["fname"] = DD + MM + YY + "N" + curve + ".csv"
        df_wide["date"] = date

        # Move identifying columns to the front
        df_wide = df_wide[["date", "fname", "curve"] + [c for c in df_wide.columns if c not in ["date", "fname", "curve"]]]

        all_rows.append(df_wide)

        del file
        gc.collect()

    # Combine all rows for this channel
    df_final = pd.concat(all_rows, ignore_index=True)

    # Save output CSV
    name = f"{date}{suffix}_{x}_calibration_stats.csv"
    fname = os.path.join(folder, name)
    df_final.to_csv(fname, index=True)

    print(f"{x} done")

# 2) Downsample and convert to CSV

In [ ]:
#Fill in filepath, date, file selections, and desired compression

#File path is assumed to lead to subfolders of dates in the format YY_MM_DD and end in /
#Only selected files will be processed; an empty list ([]) will be treated as files being selected
#select desired compression in Hz to down sample to
#"1" is treated as no down sampling
#we used 2500 hz for unzipping analysis and 2000 hz for dynamic analysis
directory=""
date = ''
selections = []
compress = 2500 

folder = directory+date

#Set constant
suffix = '.csv'
if compress != 1:
    sampling = ' down sampled to ' + str(compress) + ' Hz'
else:
    sampling = ' down sampled to ' + str(compress) + 'x'

#gather list of files
file_list = select_files(date, 'h5', selections=selections, directory=directory)

i=1

#loop through h5s and extract downsampled info
for filename in file_list:
    
    #downsample info and save to a dataframe
    if compress != 1:
        mt, mf1x, mf1y, mp1x, mf2x, mf2y, mp1y, p2x, p2y = h5_extract(filename, compress)
    else:
        mt, mf1x, mf1y, mp1x, mf2x, mf2y, mp1y, p2x, p2y = h5_extract(filename)

    #handles issues in some versions of Lumicks software where position data from trap 2 is exported weirdly
    if mt.size == 0:
        continue
    mp2x = [p2x[0]] * len(mp1x)
    mp2y = [p2y[0]] * len(mp1x)
    
    data = {'time (seconds)' : mt,
       'force 1x (pN)' : mf1x,
        'force 1y (pN)' : mf1y,
        'trap_position 1x': mp1x,
        'force 2x (pN)' : mf2x,
        'force 2y (pN)' : mf2y,
        'trap_position 1y': mp1y,
        'trap_position 2x': mp2x,
        'trap_position 2y': mp2y,}
    df = pd.DataFrame.from_dict(data)
    
    #make new file name
    curve = extract_curve_number(filename)
    new_name = date + ' FD Curve ' + curve + sampling + suffix
    new_file_name = os.path.join(folder, new_name)
    
    #save to csv
    df.to_csv(new_file_name)

    del filename
    gc.collect()

print('Done!')

# 3) Subtract baseline

In [ ]:
#If present, subtract the force baseline from the data
#This is done by fitting the baseline file to a 7th order polynomial (at Lumick's recomendation), and subtracting
#the resulting function from the data. 
#Whether or not a baseline file exists, the first 200-400 points of the data are fit to a line with slope 0 and used to shift
#the data to start at 0.1 pN to "zero" the force.

#Fill in filepath, date, file selections, and whether files are from dynamic traces

#File path is assumed to lead to subfolders of dates in the format YY_MM_DD and end in /
#Only selected files will be processed; an empty list ([]) will be treated as files being selected
#Set dynamic = False for unzipping and True for dynamic traces containing validation unzipping; changes which segment is used for correction

###Assumes there is a subfolder titled "baseline" containing downsampled baseline curves and 
#a CSV titled "baselines.csv" with a column for each baseline
#with the first row being curve numbers of baselines and below each baseline, listing the curve numbers of files to map to that baseline
#e.g. 1 4 8     
#     2 7 9
#     3
#maps curves 2 and 3 to baseline curve 1, 7 to baseline 4, and 9 to baseline 8

directory=""
date = ''
selections = []
dynamic = False

folder = directory+date

#gather files
file_list = select_files(date, 'down', selections = selections, directory=directory)

# Loop through files
for f in file_list:
    
    #import data and create new column
    df = pd.read_csv(f)
    df['force baseline 1x'] = df['force 1x (pN)']
    df['force baseline 2x'] = df['force 2x (pN)']
    curve = extract_curve_number(f)
    
    #find baseline if present; can set to -1 to skip
    curve_base = (has_baseline(folder, curve))
    
    #if present, remove baseline
    if curve_base != -1:

        # Extract baseline and fit a 7th-order polynomial (based on Lumicks recomendation) to each force (1x, 1y, 2x, 2y)
        file_list2 = select_files((date+'/baseline/'), 'down', selections = [curve_base], directory=directory)
        base = pd.read_csv(file_list2[0])
        x = base[base['trap_position 1x'] > 2.3]['trap_position 1x']
        y1x = base[base['trap_position 1x'] > 2.3]['force 1x (pN)']
        y2x = base[base['trap_position 1x'] > 2.3]['force 2x (pN)']
        x_data = df['trap_position 1x']
        
        coeffs1x = np.polyfit(x, y1x, 7)
        coeffs2x = np.polyfit(x, y2x, 7)
        
        # Create a polynomial function from the coefficients for each force
        poly1x = np.poly1d(coeffs1x)
        poly2x = np.poly1d(coeffs2x)

        #UNCOMMENT BLOCK TO PLOT CORRECTED BASELINE AS TEST
        # fix_1x = y1x - poly1x(x)
        # #fix_1y = y1y - poly1y(x)
        # fix_2x = y2x - poly2x(x)
        # #fix_2y = y2y - poly2y(x)
        
        # fig, ax = plt.subplots(2, 1, figsize=(8, 6))
        # ax[0].plot(x, y1x)
        # ax[0].plot(x, fix_1x)
        # ax[0].plot(x, poly1x(x))
        # ax[0].set_title('baseline force 1x')
        # ax[0].legend(['raw', 'corrected', 'fit'])
        # ax[0].set_xlabel('trap position (um)')
        # ax[0].set_ylabel('force (pN)')
        # # ax[0, 1].plot(x, y1y)
        # # ax[0, 1].plot(x, fix_1y)
        # # ax[0, 1].set_title('1y')
        # ax[1].plot(x, y2x)
        # ax[1].plot(x, fix_2x)
        # ax[1].plot(x, poly2x(x))
        # ax[1].set_title('baseline force 2x')
        # ax[1].set_xlabel('trap position (um)')
        # ax[1].set_ylabel('force (pN)')
        # # ax[1, 1].plot(x, y2y)
        # # ax[1, 1].plot(x, fix_2y)
        # # ax[1, 1].set_title('2y')
        # plt.tight_layout()

        #baseline correct data
        df['force baseline 1x'] = df['force 1x (pN)'] - poly1x(df['trap_position 1x'])
        df['force baseline 2x'] = df['force 2x (pN)'] - poly2x(df['trap_position 1x'])
    else:
        print('no baseline')

    #fit first 400 points to a line with slope 0 to "zero" to 0.1 pN; can change number of points if desired
    points=399
    m = 0
    new_min = 0.1
    def line(x, b):
        return m * x + b  # Only fit m

    if not dynamic:
        params, _ = curve_fit(line, df['trap_position 1x'][0:points], df['force baseline 1x'][0:points])
        b1x = params[0]
        params, _ = curve_fit(line, df['trap_position 1x'][0:points], df['force baseline 2x'][0:points])
        b2x = params[0]
    else:
        curve = extract_curve_number(f)
        valleysx, valleysy, num_peaks, peak_dict = find_peaks(df, date, curve, cycle=-1, var = 'trap', peak_dict=None,
                                                             sign = '-', num_peaks = -1, interval = 1000)
        start=valleysx[-1]
        params, _ = curve_fit(line, df['trap_position 1x'][start:start+points], df['force baseline 1x'][start:start+points])
        b1x = params[0]
        params, _ = curve_fit(line, df['trap_position 1x'][start:start+points], df['force baseline 2x'][start:start+points])
        b2x = params[0]

    df['force baseline 1x'] = df['force baseline 1x'] - b1x + new_min
    df['force baseline 2x'] = df['force baseline 2x'] - b2x + new_min

    #UNCOMMENT BLOCK TO PLOT CORRECTED DATA AS TEST
    # fig, ax = plt.subplots(2, 1, figsize=(8, 6))
    # ax[0].plot(df['trap_position 1x'], df['force 1x (pN)'].rolling(window=10).mean())
    # ax[0].plot(df['trap_position 1x'], df['force baseline 1x'].rolling(window=10).mean())
    # ax[0].set_title('Unzipping force 1x (steered trap)')
    # ax[0].set_xlabel('trap position (um)')
    # ax[0].set_ylabel('force (pN)')
    # ax[0].legend(['raw', 'corrected'])
    # # ax[0, 1].plot(df['trap_position 1x'], df['force 1y (pN)'].rolling(window=10).mean())
    # # ax[0, 1].set_title('1y')
    # ax[1].plot(df['trap_position 1x'], df['force 2x (pN)'].rolling(window=10).mean())
    # ax[1].plot(df['trap_position 1x'], df['force baseline 2x'].rolling(window=10).mean())
    # ax[1].set_title('Unzipping force 2x (stationary trap)')
    # ax[1].set_xlabel('trap position (um)')
    # ax[1].set_ylabel('force (pN)')
    # # ax[1, 1].plot(df['trap_position 1x'], df['force 2y (pN)'].rolling(window=10).mean())
    # # ax[1, 1].set_title('2y')
    
    # # ax[0].plot(df['trap_position 1x'], df['force baseline 1y'].rolling(window=10).mean())
    # # ax[1].plot(df['trap_position 1x'], df['force baseline 2y'].rolling(window=10).mean())
    # plt.tight_layout()
    # #plt.close('all')

    #save file
    df.to_csv(f)
    print('curve ' + curve + ' done')

    del f
    gc.collect()
    
print('Done!')

# 4) Calculate extension

In [ ]:
#turn trap position into extension utilizing the trap stiffnesses and baseline-corrected forces

#Fill in filepath, date, file selections and suffix (if desired to add to distinguish conditions from same date)

#File path is assumed to lead to subfolders of dates in the format YY_MM_DD and end in /
#Only selected files will be processed; an empty list ([]) will be treated as files being selected
directory = ""
date = ''
selections = []
suffix = ''

#gather files
file_list = select_files(date, 'down', selections = selections, directory=directory)

# Loop through files
for f in file_list:
    df = pd.read_csv(f)

    #extract parameters from calibration
    curve = int(extract_curve_number(f))
    k1x = extract_stiffness(date, curve, '1x', directory=directory, var='kappa', suffix=suffix)*1000
    k2x = extract_stiffness(date, curve, '2x', directory=directory, var='kappa', suffix=suffix)*1000

    #correct for bead displacement with stiffnesses and forces
    df['extension'] = df['trap_position 1x'] - (df['force baseline 1x']/k1x) - (df['force baseline 2x']/k2x) #- r1 - r2
    df['force average'] = (df['force baseline 1x'] + df['force baseline 2x'])/2

    df.to_csv(f)
    print('curve ' + str(curve) + ' done')

    #UNCOMMENT BLOCK TO PLOT CORRECTED DATA AS TEST
    # #plt.plot(df['extension'], df['force baseline 1x'])
    # plt.plot(df['trap_position 1x'], df['force baseline 2x'])
    # plt.plot(df['extension'], df['force baseline 2x'])
    # plt.xlabel('microns')
    # plt.ylabel('Force (pN)')
    # #plt.plot(df['extension'], df['force average'])
    # plt.legend(['Force 2x vs trap position 1', 'Force 2x vs DNA extension'])
    # #plt.close('all')

    del f
    gc.collect()
    
print('Done!')

## Optional: Rename files for Matlab pipeline

In [ ]:
#Downstream MATLAB pipeline expects a different file naming scheme as input
#This function copies data into a new file with appropriate naming
#DDMMYYNXX (e.g "24_04_10 FD Curve 73 down sampled to 2500 Hz.csv" to be 100424N73)

#Fill in filepath, date, and file selections

#File path is assumed to lead to subfolders of dates in the format YY_MM_DD and end in /
#Only selected files will be processed; an empty list ([]) will be treated as files being selected
#Change to cycles = True if converting files that were split into cycles; file will be saved as DDMMYYNXXCC (where CC is cycle number)
directory=""
date = ''
selections = []
cycles = False

suffix = ".csv"
folder = directory+ date

#gather files
file_list = select_files('', 'down', selections = selections, cycles=cycles, directory=folder)

#If saving cycles, only keeps F cycles (forward) by default
fi2 = []
for i in range(0, len(file_list)):
    if "R down" not in file_list[i]:
            fi2.append(file_list[i])

dates = date.split('_')
YY = dates[0]
MM = dates[1]
DD = dates[2]

#Loop through files and get curve and cycle
for f in fi2:
    curve = extract_curve_number(f)
    if cycles:
        cycle = extract_cycle_number(f)
    else:
        cycle = ""

    df = pd.read_csv(f)
    new_name = DD + MM + YY + 'N' + curve + cycle + suffix
    # new_name = DD + MM + YY + 'N' + curve + suffix
    new_file_name = os.path.join(folder, new_name)
    
    #save to csv
    df.to_csv(new_file_name)
    
    print(DD + MM + YY + 'N' + curve + cycle + " Done!")
    #print(DD + MM + YY + 'N' + curve + " Done!")
print("Done!")

# Optional: Plot curves for figures

In [ ]:
#File in filepath, date, file selections, whether the data is full unzipping or cycles, the type of desired plot,
#whether a legend and title are desired, and if you want data smoothed

#File path is assumed to lead to subfolders of dates in the format YY_MM_DD and end in /
#Only selected files will be processed; an empty list ([]) will be treated as files being selected
#Change to cycles = True if plotting files that were split into cycles
#Plot can be set to "d" (force vs extension), "t" (force vs time), "d/t" (side by side subplots of "d" and "t"), 
#or "d/t/p" (side by side subplots of "d", "t", and extension vs time)
#legend can be set to True (curve numbers), False (none), or a list of condition names
#ftitle species a figure title
#smooth can be false or an integer specifying degree of smoothness (currently only works for "d" and "t" not "d/t" or "d/t/p")
directory=""
date = ''
selections = []
cycles = False
plot = 'd'
legend = True
ftitle = ''
smooth = False

fi = select_files(date, 'down', directory, selections, cycles=cycles)

fi2 = []
for i in range(0, len(fi)):
    if "R down" not in fi[i]:
            fi2.append(fi[i])

fig = plot_curves(fi2, plot=plot, legend=legend, ftitle=ftitle, smooth=smooth)

# Optional: Split into cycles

In [ ]:
#Fill in filepath, date, and file selections

#File path is assumed to lead to subfolders of dates in the format YY_MM_DD and end in /
#Assumes there is a subfolder titled "cycles"
#Only selected files will be processed; an empty list ([]) will be treated as files being selected
directory = ""
date = ''
selections = []

folder = directory+date + '/cycles/'

#gather files
file_list = select_files(date, 'down', selections = selections, directory=directory)

#Loop through files
for f in file_list:
    
    #import data
    df = pd.read_csv(f)
    curve = extract_curve_number(f)
    
    cycle = str(-1)
    
    #find position peaks and valleys
    peak_dict = None
    peaksx, peaksy, num_peaks, peak_dict = find_peaks(df, date, curve, cycle, var = 'trap', peak_dict=peak_dict,
                                                      sign = '+', num_peaks = -1, interval = 1000)
    valleysx, valleysy, num_peaks, peak_dict = find_peaks(df, date, curve, cycle, var = 'trap', peak_dict=peak_dict,
                                                          sign = '-', num_peaks = num_peaks, interval = 1000)
    
    if len(peaksx) == 0:
        continue
    
    #loop through cycles
    for i in range(0, len(peaksx)):
        
        #make new file names
        stri = str(i+1)
        if i+1 <10:
            stri = '0' + stri
        
        curve = extract_curve_number(f)
        
        suffix = '.csv'
        loc1 = f.find('down')
        loc2 = f.find('x.csv')
        new_name = date + ' FD Curve ' + curve + ' Cycle ' + stri 
        new_nameF = new_name + " F" + ' down sampled to 2500 Hz' + suffix
        new_nameR = new_name + " R" + ' down sampled to 2500 Hz' + suffix
        
        new_file_nameF = os.path.join(folder, new_nameF)
        new_file_nameR = os.path.join(folder, new_nameR)
        
        #split cycles and save F to csv
        df_f = df.iloc[valleysx[i]:peaksx[i], :]
        df_f.to_csv(new_file_nameF)
        
        #split cycles and save R to csv
        if not (len(peaksx) == len(valleysx) and i == len(peaksx)-1):
            df_r = df.iloc[peaksx[i]:valleysx[i+1], :]
            df_r.to_csv(new_file_nameR)